<a href="https://colab.research.google.com/github/jads-nl/public-lectures/blob/main/nlp/nlp-pipeline-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U pendulum spacy

     |████████████████████████████████| 163kB 4.2MB/s 
     |████████████████████████████████| 12.8MB 280kB/s 
     |████████████████████████████████| 491kB 32.6MB/s 
     |████████████████████████████████| 1.1MB 49.8MB/s 
     |████████████████████████████████| 9.1MB 28.4MB/s 
     |████████████████████████████████| 460kB 36.4MB/s 
     |████████████████████████████████| 122kB 43.1MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=8e7d2c8281822e8e7d06419ff8227d2a0dce966fdad8ebe296f5b627db25c190
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: smart-open 4.2.0
    Uninstalling smart-open-4.2.0:
      Successfully uninstalled smart-open-4.2.0
  Found existing installation: srsly 1.0.5
    Uninst

In [ ]:
!python -m spacy download nl_core_news_lg

2021-03-25 07:38:54.975597: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 573.2MB 20kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_lg')


In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier, BalancedBaggingClassifier    
import pandas as pd
import pendulum
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.tree import DecisionTreeClassifier
import spacy


# only use 2019 data as example
df = pd.read_parquet("https://github.com/jads-nl/public-lectures/blob/main/nlp/data/dutch-restaurant-reviews-per-year/reviewYear%3D2019/058d741d776d45f18e0ccc51f71173dc.parquet?raw=true")

# initatie spacy model
nlp = spacy.load("nl_core_news_lg")

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported fro

# Dutch Restaurant reviews

## Objective
Predict a 'detractor' such that restaurant owner can look-up interesting (negative) feedback and act upon that.

## Data preparation

### Select main columns

In [ ]:
reviews = df.loc[:, ['restoId', 'reviewerId', 'reviewerFame', 'reviewerNumReviews', 'reviewText']].copy()

### Format date columns

In [ ]:
def parse_date(date):
    return pendulum.from_format(date, fmt="D MMM YYYY", locale="nl")

reviews["reviewDate"] = df.reviewDate.apply(parse_date).dt.date

### Format numerical columns

In [ ]:
def clean_price(string):
    "Remove euro sign and whitespace in price"
    if string:
        return float(string.split(" ")[-1])
    else:
        return 0


reviews["avgPrice"] = df["avgPrice"].fillna(0).apply(clean_price)


# numerical columns have comma as decimal seperator --> cast to floats
numerical_cols = [
    "scoreFood",
    "scoreService",
    "scoreDecor",
    "reviewScoreOverall",
    "scoreTotal",
]
for col in numerical_cols:
    reviews[col] = pd.to_numeric(df[col])

### Format ordinal columns

In [ ]:
map_scores = {
    "waitingTimeScore": {
        None: 0,
        "Hoog tempo": 1,
        "Kort": 2,
        "Redelijk": 3,
        "Kan beter": 4,
        "Lang": 5,
    },
    "valueForPriceScore": {
        None: 0,
        "Erg gunstig": 1,
        "Gunstig": 2,
        "Redelijk": 3,
        "Precies goed": 4,
        "Kan beter": 5,
    },
    "noiseLevelScore": {
        None: 0,
        "Erg rustig": 1,
        "Rustig": 2,
        "Precies goed": 3,
        "Rumoerig": 4,
    },
    "reviewerFame": {
        None: 0,
        "Proever": 1,
        "Fijnproever": 2,
        "Expertproever": 3,
        "Meesterproever": 4
    }
}

for col in map_scores.keys():
    reviews[col] = (
        df[col].apply(lambda x: map_scores[col].get(x, None)).astype("Int64")
    )

## Text pre-processing

### Filter reviews that are short or in process

In [ ]:
def validate_review(review):
    if review == '- Recensie is momenteel in behandeling -' or len(review) < 4:
        return False
    else:
        return True
    

reviews['is_valid'] = reviews.reviewText.apply(validate_review)

### Add simple features

In [ ]:
reviews['review_char_length_'] = df.reviewText.apply(lambda x: len(x))

### Tokenize and create Document-Term Matrix

We will use [pandas sparse data structures](https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html) to save memory. Note cell below takes about 9 minutes to complete.

In [ ]:
%%time
def tokenize_simple(text):
    """Tokenizer returning lowercase tokens with no stop words, no punctuation and no words with encoding errors"""
    doc = nlp(text)
    return [token.lower_ for token in doc if not (token.is_stop or token.is_punct or ("\\" in token.lower_))]


# some abbreviations aren't in spaCy's default Dutch stopwords list, so we add them
nlp.Defaults.stop_words.update(['n', 't'])

count_vectorizer = CountVectorizer(tokenizer=tokenize_simple, stop_words=nlp.Defaults.stop_words, ngram_range=(1,1))
dtm = pd.DataFrame.sparse.from_spmatrix(count_vectorizer.fit_transform(reviews.reviewText), columns=count_vectorizer.get_feature_names())

CPU times: user 10min 38s, sys: 918 ms, total: 10min 39s
Wall time: 10min 42s


We will only keep words in the DTM that occur twice or more over all the reviews. This reduces the width of the DTM.

In [ ]:
token_filter = (dtm.sum() > 2)
token_filter[token_filter == True]
print(f"Full DTM: {dtm.shape}")
print(f"Filtered DTM: {dtm.loc[:, token_filter].shape}")

Full DTM: (47048, 32823)
Filtered DTM: (47048, 11050)


## Binary classification: `is_detractor`

### Define Y

In [ ]:
reviews["is_detractor"] = reviews.reviewScoreOverall.apply(lambda x: True if x <= 6 else False)

### Train-test split

(Cell below takes about two minute).

In [ ]:
%%time
X = reviews[reviews.is_valid].drop(columns=["reviewDate", "reviewText", "scoreFood", "scoreService", "scoreDecor", "reviewScoreOverall", "scoreTotal", "is_detractor"])
y = reviews[reviews.is_valid].is_detractor

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X,y):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    dtm_train, dtm_test = dtm.loc[reviews.is_valid, token_filter].iloc[train_index, :], dtm.loc[reviews.is_valid, token_filter].iloc[test_index, :]

CPU times: user 1min 55s, sys: 3.26 s, total: 1min 58s
Wall time: 1min 59s


### BalancedBaggingClassifier - without DTM



In [ ]:
%%time
clf1 = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)
clf1.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

CPU times: user 837 ms, sys: 21 ms, total: 858 ms
Wall time: 875 ms


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


We will skip fine-tuning the model, our purpose is to compare it with a model that adds text. Using the balanced accuracy to compare, which is defined as the average of recall obtained on each class.

In [ ]:
balanced_accuracy_score(y_test, clf1.predict(X_test)).round(2)

0.72

### BalancedBaggingClassifier - with DTM

(Cell below takes about 50 minutes on Google Colab)

In [ ]:
%%time
clf2 = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)
clf2.fit(dtm_train.join(X_train), y_train) 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

CPU times: user 50min 11s, sys: 30.7 s, total: 50min 42s
Wall time: 50min 46s


In [ ]:
balanced_accuracy_score(y_test, clf2.predict(dtm_test.join(X_test))).round(2)

0.83

## Closing remarks

We have illustrated how a simple bag-of-words model can add to the performance of a classifier that uses structured data. We haven't optimized the modeling at all, but done a simple like-to-like comparison with the same parameters.

Note that working with text requires more engineering: you need to make decisions about how to store and process the data because it can quickly expand beyond the memory of your (virtual) machine. Even with this simple model, we have used over 700 features from a truncated document-term matrix.